In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data= pd.read_csv('D:/Users/NITIN VERMA/Desktop/DL-NLP/MED PROJECT/data/medical_new.csv')
data.head()

,Unnamed: 0,medical_specialty,transcription,medical_label,word2vec_embed,fasttext_label
0,3,cardiovascular_pulmonary,d m mode leave atrial enlargement left atrial ...,0,[-3.9130253e-01 1.6925055e+00 -9.7060895e-01 ...,__label__cardiovascular_pulmonary d m mode lea...
1,4,cardiovascular_pulmonary,left ventricular cavity size wall thickness ap...,0,[-5.44876158e-01 1.30335271e+00 -1.53507555e+...,__label__cardiovascular_pulmonary left ventric...
2,7,cardiovascular_pulmonary,d echocardiogram multiple view heart great ves...,0,[-0.40531728 0.8317133 -1.0865903 0.296900...,__label__cardiovascular_pulmonary d echocardio...
3,9,cardiovascular_pulmonary,description normal cardiac chamber size normal...,0,[-0.3478441 0.9546592 -1.0535976 1.201930...,__label__cardiovascular_pulmonary description ...
4,11,cardiovascular_pulmonary,d study mild aortic stenosis widely calcify mi...,0,[ 0.27329454 1.1477208 -0.88355297 1.073992...,__label__cardiovascular_pulmonary d study mild...


In [31]:
# since word2vec have been converted into string. we need to create them again
import spacy
nlp= spacy.load('en_core_web_lg')

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_lg' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [33]:
def getvector(text):
    doc= nlp(text)
    return doc.vector

In [37]:
data['word2vec_embed']= data['transcription'].apply(getvector)

In [117]:
x_train, x_test, y_train, y_test= train_test_split(data.word2vec_embed, data.medical_label, stratify= data.medical_label, test_size= 0.2, random_state= 5354)

In [118]:
x_train_np= np.stack(x_train)
x_test_np= np.stack(x_test)

In [119]:
#handle -ve values
m= MinMaxScaler()
x_train_new= m.fit_transform(x_train_np)
x_test_new= m.fit_transform(x_test_np)

In [120]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

naive bayes model

In [121]:
model= MultinomialNB()
model.fit(x_train_new, y_train)

MultinomialNB()

In [122]:
model.score(x_test_new, y_test)

0.6166666666666667

In [123]:
y_pred= model.predict(x_test_new)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.43      0.54        56
           1       0.71      0.64      0.68        39
           2       0.59      0.56      0.58        34
           3       0.62      0.80      0.70        61
           4       0.51      0.62      0.56        50

    accuracy                           0.62       240
   macro avg       0.63      0.61      0.61       240
weighted avg       0.63      0.62      0.61       240



random forest model

In [124]:
from sklearn.ensemble import RandomForestClassifier

model= RandomForestClassifier()
model.fit(x_train_new, y_train)

RandomForestClassifier()

In [125]:
model.score(x_test_new, y_test)

0.5708333333333333

In [126]:
y_pred= model.predict(x_test_new)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.41      0.49        56
           1       0.72      0.67      0.69        39
           2       0.62      0.44      0.52        34
           3       0.71      0.75      0.73        61
           4       0.35      0.54      0.43        50

    accuracy                           0.57       240
   macro avg       0.60      0.56      0.57       240
weighted avg       0.60      0.57      0.57       240



## SMOTE ANALYSIS

In [127]:
def smote(dataframe, model):
    from imblearn.over_sampling import SMOTE 
    
    #features and target
    x= dataframe['word2vec_embed'] 
    x_np= np.stack(x)
    m= MinMaxScaler()

    x_new= m.fit_transform(x_np)

    y= dataframe['medical_label']

    #smote apply
    smote = SMOTE()
    X_sm, y_sm = smote.fit_resample(x_new,y)

    #train test split
    x_train, x_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=5354, stratify=y_sm)
    model.fit(x_train, y_train)
    print(model.score(x_test, y_test)) 
    
    y_pred= model.predict(x_test)
    print(classification_report(y_test, y_pred))

Naive bayes model

In [128]:
smote(data, MultinomialNB())

0.6270627062706271
              precision    recall  f1-score   support

           0       0.73      0.39      0.51        61
           1       0.79      0.77      0.78        60
           2       0.52      0.72      0.61        60
           3       0.66      0.74      0.70        61
           4       0.52      0.52      0.52        61

    accuracy                           0.63       303
   macro avg       0.64      0.63      0.62       303
weighted avg       0.64      0.63      0.62       303



random forest model

In [129]:
smote(data, RandomForestClassifier())

0.6600660066006601
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        61
           1       0.83      0.88      0.85        60
           2       0.66      0.68      0.67        60
           3       0.78      0.74      0.76        61
           4       0.31      0.33      0.32        61

    accuracy                           0.66       303
   macro avg       0.66      0.66      0.66       303
weighted avg       0.66      0.66      0.66       303

